In [1]:
import os
os.chdir('../')

from medclip import MedCLIPModel, MedCLIPVisionModelViT
from medclip import MedCLIPProcessor
from medclip import PromptClassifier

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# init models
processor = MedCLIPProcessor()
ckpt_path = "./data/MedCLIP/checkpoints/vision_text_pretrain/25000"
checkpoint_path = "/Users/sandradening/Documents/Dokumente_Sandra/Master_Studium/3_Semester/VLM_Seminar/Code/pretrained/medclip-vit"

model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT, checkpoint=checkpoint_path)
clf = PromptClassifier(model, ensemble=True)
clf.cuda()

/Users/sandradening/anaconda3/envs/CAMP/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
/Users/sandradening/Documents/Dokumente_Sandra/Master_Studium/3_Semester/VLM_Seminar/GitHUB_MEdclip/MedCLIP/medclip/modeling_medclip.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mod

RuntimeError: Error(s) in loading state_dict for MedCLIPModel:
	Unexpected key(s) in state_dict: "text_model.model.embeddings.position_ids". 

In [3]:
# prepare input image
from PIL import Image
image = Image.open('./example_data/view1_frontal.jpg')
inputs = processor(images=image, return_tensors="pt")

# prepare input prompt texts
from medclip.prompts import generate_chexpert_class_prompts, process_class_prompts

cls_prompts = process_class_prompts(generate_chexpert_class_prompts(n=10))
inputs['prompt_inputs'] = cls_prompts

sample 10 num of prompts for Atelectasis from total 210
sample 10 num of prompts for Cardiomegaly from total 15
sample 10 num of prompts for Consolidation from total 192
sample 10 num of prompts for Edema from total 18
sample 10 num of prompts for Pleural Effusion from total 54


In [4]:
output = clf(**inputs)
print(output)

{'logits': tensor([[0.5154, 0.4119, 0.2831, 0.2441, 0.4588]], device='cuda:0',
       grad_fn=<StackBackward0>), 'class_names': ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']}
